# Comparing Proportional Selection and SUS Selection
In this notebook, I will compare the results of using proportional selection and SUS selection. 

In [1]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from distributed import Client, LocalCluster


from leap_ec.algorithm import generational_ea
from leap_ec.individual import Individual
from leap_ec.representation import Representation
import leap_ec.ops as ops
from leap_ec import probe
from leap_ec.global_vars import context
from leap_ec.distrib import synchronous


from leap_ec.real_rep.problems import LangermannProblem
from leap_ec.real_rep.ops import mutate_gaussian
from leap_ec.real_rep.initializers import create_real_vector

## Define Parameters

In [2]:
MAX_GEN = 1000    # number of generations per iteration
POP_SIZE = 10     # population size
NUM_ITER = 100    # total number of iterations per selection operator
BOUNDS = [[0, 10]]*2  # bounds for LangermannProblem
CONFIDENCE = 0.99 # confidence interval for plot

## Run Proportional and Track Stats

In [ ]:
with Client(LocalCluster(n_workers=os.cpu_count(), threads_per_worker=1, processes=True)) as client:
    with open('./proportional_stats.csv', 'w') as prop_file:
        prop_stats_probe = probe.FitnessStatsCSVProbe(stream=prop_file, context=context)
        for i in tqdm(range(NUM_ITER)):
            prop_stats_probe.bsf_ind = None
            ea = generational_ea(max_generations=MAX_GEN, pop_size=POP_SIZE,
                                 problem=LangermannProblem(maximize=True),
                                 init_evaluate=synchronous.eval_population(client=client),
                                 representation=Representation(
                                     initialize=create_real_vector(bounds=BOUNDS)
                                 ),
                                 pipeline=[
                                     ops.proportional_selection(offset='pop-min'),
                                     ops.clone,
                                     ops.uniform_crossover(p_swap=0.2),
                                     mutate_gaussian(std=0.5, expected_num_mutations='isotropic'),
                                     synchronous.eval_pool(size=POP_SIZE,client=client),
                                     prop_stats_probe
                                 ])
            list(ea);

  4%|▍         | 4/100 [01:42<40:43, 25.45s/it]

## Run SUS and Track Stats

In [ ]:
sus_file = open('./SUS_stats.csv', 'w')
sus_stats_probe = probe.FitnessStatsCSVProbe(stream=sus_file, context=context)
for i in tqdm(range(NUM_ITER)):
    sus_stats_probe.bsf_ind = None
    ea = generational_ea(max_generations=MAX_GEN, pop_size=POP_SIZE,
                         problem=LangermannProblem(maximize=True),
                         representation=Representation(
                             initialize=create_real_vector(bounds=BOUNDS)
                         ),
                         pipeline=[
                             ops.sus_selection(n=4, offset='pop-min'),
                             ops.clone,
                             ops.uniform_crossover,
                             mutate_gaussian(std=0.5, expected_num_mutations='isotropic'),
                             ops.evaluate,
                             ops.pool(size=POP_SIZE),
                             sus_stats_probe
                         ])
    list(ea);
sus_file.close()

## Data Manipulation

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

# read in data and calculate averages
prop_df = pd.read_csv('proportional_stats.csv')
sus_df = pd.read_csv('SUS_stats.csv')

prop_df_step = prop_df.groupby('step').mean()
sus_df_step = sus_df.groupby('step').mean()

sus_df_step.to_csv('SUS_avg_stats.csv')
prop_df_step.to_csv('prop_avg_stats.csv')

prop_bsf = prop_df_step[' bsf'].values
sus_bsf = sus_df_step[' bsf'].values

## Compute Confidence Intervals

In [ ]:
# create lists to store +/- (up/down) confidence intervals for each generation
prop_conf_up = []
prop_conf_down = []
sus_conf_up = []
sus_conf_down = []
for i in range(MAX_GEN):
    prop_step_data = prop_df[prop_df['step'] == i][' bsf'].values
    sus_step_data = sus_df[sus_df['step'] == i][' bsf'].values
    prop_int_up, prop_int_down = stats.t.interval(CONFIDENCE, len(prop_step_data)-1, loc=prop_bsf[i], scale=stats.sem(prop_step_data)) 
    sus_int_up, sus_int_down = stats.t.interval(CONFIDENCE, len(sus_step_data)-1, loc=sus_bsf[i], scale=stats.sem(sus_step_data))
    prop_conf_up.append(prop_int_up)
    prop_conf_down.append(prop_int_down)
    sus_conf_up.append(sus_int_up)
    sus_conf_down.append(sus_int_down)
    
print(prop_conf_up[0:10])
print(prop_conf_down[0:10])
print(sus_conf_up[0:10])
print(sus_conf_down[0:10])

## Plot Result

In [ ]:
steps = list(range(MAX_GEN))
plt.plot(steps, prop_bsf, color='blue', label='Proportional')
plt.fill_between(steps, prop_conf_up, prop_conf_down, alpha=0.25)
plt.plot(steps, sus_bsf, color='red', label='SUS')
plt.fill_between(steps, sus_conf_up, sus_conf_down, alpha=0.25)
plt.legend()
plt.ylabel('Best-so-far fitness')
plt.xlabel('Generation')
plt.title('Average Best-so-far Fitness vs Generations')
plt.savefig(f'prop_vs_sus_bsf_conf{CONFIDENCE}_langermann.png')

plt.show()
